#  Models

## Imports 

In [ ]:
#Directory Libraries
import sys, os
import glob
from os.path import join
#plotting libraries
import matplotlib.pylab as plt
from tqdm.notebook import tqdm
import seaborn as sns
#pre-processing 
import cv2
from numpy.random import seed
import pandas as pd
from PIL import Image
import numpy as np
#tensorflow
import tensorflow
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.backend as K
from tensorflow.keras import layers,Model
from tensorflow.keras import initializers, layers
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras import layers, models, optimizers, Input
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
#tensorflow.device("GPU:0")
#parallel computing
from joblib import Parallel, delayed
import argparse
#Ignore unwanted warnings
import warnings
warnings.filterwarnings('ignore')
seed(101)
%matplotlib inline
#Make sure that plots are colorblindfriendly
plt.style.use("seaborn-colorblind")


In [ ]:
Processed_datasets = {
    'train_dir': '../input/capstone-dataset/base__dir/train_dir',
    'val_dir': '../input/capstone-dataset/base__dir/val_dir',
    'test_dir':'../input/capstone-dataset/base__dir/test_dir'
    
}


In [ ]:
df_train = pd.read_csv("../input/capstone-dataset/base__dir/train_data.csv")
df_val = pd.read_csv("../input/capstone-dataset/base__dir/val_data.csv")
df_test = pd.read_csv("../input/capstone-dataset/base__dir/test_data.csv")


In [ ]:
num_train_samples = len(df_train)
num_val_samples = len(df_val)
num_test_samples = len(df_test)

train_batch_size = 26
val_batch_size = 26
test_batch_size = 26

train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)
test_steps = np.ceil(num_test_samples / test_batch_size)


In [ ]:
datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1./255)


train_gen = datagen.flow_from_directory(Processed_datasets['train_dir'],
                                        target_size=(244,244),
                                        batch_size=train_batch_size,
                                        class_mode='categorical')

val_gen = test_datagen.flow_from_directory(Processed_datasets['val_dir'],
                                        target_size=(244,244),
                                        batch_size=val_batch_size,
                                        class_mode='categorical')
test_gen = test_datagen.flow_from_directory(Processed_datasets['test_dir'],
                                        target_size=(244,244),
                                        batch_size=test_batch_size,
                                        class_mode='categorical')

In [ ]:
model=Xception(include_top=False,
                 weights='imagenet',
                 input_shape=(244,244,3),pooling ='avg',
                 )
model.summary()


In [ ]:
base_model = Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(244, 244, 3),
    include_top=False,pooling ='avg')  # Do not include the ImageNet classifier at the top.

In [ ]:
inputs = Input(shape=(244, 244, 3))

for layer in base_model.layers:
    layer.trainable = True

x = base_model.output
predicted = Dense(2,activation ='softmax')(x)
model_pretrain = Model(base_model.input, predicted)
model_pretrain.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
          optimizer=Adam(lr = 0.0001),
          metrics=['accuracy'])
pretraining_Xception =model_pretrain.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=50, verbose=1)


In [ ]:

plt.figure(figsize=(12,6),dpi=120)
plt.plot(pretraining_Xception.history['accuracy'], label=' Train Accuracy')
plt.plot(pretraining_Xception.history['val_accuracy'],label=' Validation Accuracy')

plt.xlabel("Epoch")
plt.ylabel("Accuracy")

plt.title("CNN Model")
plt.legend()
plt.show()

In [ ]:
model_pretrain.save("model_finetuned_xception.hdf5")

In [ ]:
import seaborn as sns
sns.set()

In [ ]:
plt.figure(figsize=(12,6),dpi=120)
plt.plot(pretraining_Xception.history['loss'], label=' Train Loss')
plt.plot(pretraining_Xception.history['val_loss'],label=' Test Loss')

plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.title("CNN Model")
plt.legend()
plt.show()

In [ ]:
predict = model_pretrain.predict_generator(test_gen,steps = test_steps)


In [ ]:
score, acc = model_pretrain.evaluate(test_gen,
                            steps = test_steps)

In [ ]:
pretraining_Xception_history = pd.DataFrame(pretraining_Xception.history)
pretraining_Xception_history.to_csv("Xception.history.csv")
